In [1]:
import os
from glob import glob
import pandas as pd

In [2]:
# path
path = os.path.abspath(r'C:\Users\USER\Guro_Psy_KJH Dropbox\1.Projects\1_anxiety_VR\3_Data')
data_path = os.path.join(path,'4_CRF','CRF_20230811.xlsx')
sheet_name = '창의도전'

In [3]:
# sub_dictionary
# CRF="C:\Users\USER\Guro_Psy_KJH Dropbox\1.Projects\1_anxiety_VR\3_Data\4_CRF\(혁신형, 창의도전)_CRF(20230811).xlsx"
# key: subject number
# value: list [group, number of session]

sub_dic = {'sub-0001':[0,2],'sub-0002':[0,2],'sub-0003':[0,2],'sub-0004':[0,2],'sub-0005':[0,1],'sub-0006':[0,2],
       'sub-0008':[0,2],'sub-0011':[0,2],'sub-0013':[0,2],'sub-0014':[0,2],'sub-0020':[0,2],'sub-0022':[0,2],
       'sub-0024':[0,2],'sub-0025':[0,2],'sub-0026':[0,2],'sub-0029':[0,2],'sub-0030':[0,1],'sub-0031':[0,2],
       'sub-0034':[0,2],'sub-0038':[0,2],'sub-0044':[0,2],'sub-0046':[0,2],'sub-0048':[0,2],'sub-0050':[0,2],
       'sub-0051':[0,2],'sub-0052':[1,1],'sub-0053':[0,2],'sub-0054':[0,2],'sub-0055':[0,2],'sub-0058':[0,2],
       'sub-0059':[0,1],
       'sub-0007':[1,4],'sub-0009':[1,4],'sub-0012':[1,4],'sub-0016':[1,4],'sub-0017':[1,4],'sub-0018':[1,4],
       'sub-0021':[1,4],'sub-0023':[1,4],'sub-0028':[1,4],'sub-0032':[1,4],'sub-0033':[1,4],'sub-0035':[1,4],
       'sub-0036':[1,4],'sub-0037':[1,4],'sub-0039':[1,4],'sub-0040':[1,4],'sub-0041':[1,4],'sub-0042':[1,4],
       'sub-0043':[1,4],'sub-0045':[1,4],'sub-0047':[1,4],'sub-0049':[1,4],'sub-0057':[1,4],'sub-0061':[1,1],
       'sub-0062':[1,4],'sub-0063':[1,4]}

cols = ['age','sex','marriage','edu_y','px_med_d','smok_1','alcohol_1','group','pdss',
       'lsas_f', 'lsas_a', 'gad', 'hads_a', 'hads_d', 'ius',
       'asi', 'bfne', 'pswq', 'ssq']

In [4]:
def modify_CRF(data_path,sub_dic, cols,sheet_name):
    """
    Generates a DataFrame with a modified structure based on specific criteria. 
    The function reads the original Excel file, processes the data based on 
    the given 'sub_dic', and restructures the DataFrame according to the 
    specified columns 'cols' and session numbers within 'sub_dic'.

    criteria

    1. data에서 cols의 8번째 까지 값들과 같은 column의 경우는 data의 index 값과 df의 index의 앞부분 (sub-00xx) 가 같으면 
    df에 data와 같은 값이 들어가도록 함
    2. data의 column들 중 cols에 해당하는 경우,(척도들) data의 index 값과 df의 index의 앞부분 (sub-00xx)이 같고 
    coulmns의 '_' 마지막 위치를 기준으로 같은 session의 값을 입력한다.

    Args:
    - data_path (string): Path to the original Excel file
    - sub_dic (dict): Dictionary with keys as subjects and values as lists containing session information
    - cols (list): List of column names to be used in the new DataFrame
    - sheet_name (string): Name of the Excel sheet to read

    Returns:
    - df (DataFrame): Modified DataFrame with structure based on 'sub_dic' and 'cols'

    Reference:
    - ref3: Information and location by file
    """
    # load original file
    data = pd.read_excel((data_path), sheet_name= sheet_name, index_col=0)

    # maker new df frame, with index from sub_dic
    new_index =[]
    for key, value in sub_dic.items():
        for i in range(1, value[1] + 1):
            new_index.append(f"{key}_ses-0{i}")

    # 새로운 DataFrame 생성
    df = pd.DataFrame(index=new_index, columns=cols)

    # data DataFrame에서 값 복사
    for idx in df.index:
        sub_key = idx.split('_ses')[0]
        ses_num = int(idx.split('-')[-1])
        if sub_key in sub_dic.keys() and sub_dic[sub_key][1] >= ses_num:
            # 처음 8개의 열은 동일한 값으로 채웁니다.
            df.loc[idx, cols[:8]] = data.loc[sub_key, cols[:8]]
            # 나머지 열에 대해서는 특정 조건을 만족하는 경우에 값을 복사합니다.
            for col in data.columns:
                position = col.rfind('_')
                if col[:position] in cols:                
                    if col.split('_')[-1] == str(ses_num):
                        df.loc[idx, col[:position] ] = data.loc[sub_key, col]
    return df




In [5]:
df = modify_CRF(data_path, sub_dic, cols, sheet_name)
df.to_csv(os.path.join(path,'CRF_230822.csv'))
